In [14]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix


## Movielens full - EDA
a) Gör en EDA för att förstå datasetet. Inkludera olika slags plots. Begränsa dig inte till frågorna nedan,
utan försök undersöka fler aspekter av datan.

* Jag börjar med att undersöka vad datasetet har för columner med df.head och df.info()
* Samt så har jag läst på "Read me" filen som tillhör datasetet

In [15]:
df_movies= pd.read_csv("Data/ml-latest/movies.csv")
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
# Jag tar df.info 
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86537 entries, 0 to 86536
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  86537 non-null  int64 
 1   title    86537 non-null  object
 2   genres   86537 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [17]:
df_ratings= pd.read_csv("Data/ml-latest/ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [18]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 1.0 GB


In [19]:
#MovieID to movie name mapping (Youtube Spencer Pao)
movie_names= df_movies.set_index('movieId')['title'].to_dict()
n_users = len(df_ratings.userId.unique())
n_items = len(df_ratings.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_users)
print("The ful rating matrix will have:", n_users*n_items,'elements.')
print('.............')
print("Number of ratings:",len(df_ratings))
print("Therefore:", len(df_ratings)/(n_users*n_items)*100, '% of the matrix is filled.') 


Number of unique users: 330975
Number of unique movies: 330975
The ful rating matrix will have: 27550028025 elements.
.............
Number of ratings: 33832162
Therefore: 0.12280264096028991 % of the matrix is filled.


In [20]:
import torch
from torch.autograd import Variable
from tqdm.notebook import tqdm


class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # Skapa användarinbäddningar
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        # Skapa objektinbäddningar
        self.item_factors = torch.nn.Embedding(n_items, n_factors)
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
    
    def forward(self, data):
        # Matrixmultiplikation
        users, items = data[:, 0], data[:, 1]
        user_embedding = self.user_factors(users)
        item_embedding = self.item_factors(items)
        return (user_embedding * item_embedding).sum(1)
    
    def predict(self, user, item):
        return self.forward(torch.tensor([[user, item]], dtype=torch.long))


In [21]:
#Creating the data loader (nececery for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

class Loader(Dataset):
    def __init__(self):
        self.ratings = df_ratings.copy()
        
        #Extract all user IDs and movie IDs
        users = df_ratings.userId.unique()
        movies = df_ratings.movieId.unique()
        
        #Unique values: index
        self.userid2idx = {i:i for i,o in enumerate(users)}
        self.movieid2idx = {i:i for i,o in enumerate(movies)}
        
        #Obtiaind continuoues ID for users and movies
        self.idx2userid = {i:o for o, i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o, i in self.movieid2idx.items()}
        
        #return the id from the index values as noted in the lambada function down below.
        #self.ratings.movieId = df_ratings.movieId.apply(lambda x: self.movieid2idx[x])
        #self.ratings.userId =df_ratings.userId.apply(lambda x: self.userid2idx[x])
        #self.ratings.movieId = df_ratings.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = df_ratings.userId.apply(lambda x: self.userid2idx.get(x, None))
        self.ratings.movieId = df_ratings.movieId.apply(lambda x: self.movieid2idx.get(x, None))


        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y =self.ratings['rating'].values
        self.x , self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors(ready for torch models.)
        
    def __getitem__ (self, index):
        return (self.x[index], self.y[index])
        
    def __len__(self):
        return len(self.ratings)
        

In [22]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        
#GPU enable if yu have a GPE...
if cuda:
    model = model.cuda()
# MSE loss
loss_fn = torch.nn.MSELoss()

#ADM optimizer
optimizer = torch.nn.MSELoss()

#Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle = True)
     

Is running on GPU False
MatrixFactorization(
  (user_factors): Embedding(330975, 8)
  (item_factors): Embedding(83239, 8)
)
user_factors.weight tensor([[0.0167, 0.0351, 0.0042,  ..., 0.0484, 0.0329, 0.0490],
        [0.0161, 0.0338, 0.0314,  ..., 0.0023, 0.0213, 0.0042],
        [0.0376, 0.0420, 0.0476,  ..., 0.0474, 0.0320, 0.0144],
        ...,
        [0.0286, 0.0420, 0.0131,  ..., 0.0426, 0.0120, 0.0037],
        [0.0383, 0.0317, 0.0439,  ..., 0.0291, 0.0081, 0.0066],
        [0.0466, 0.0217, 0.0143,  ..., 0.0327, 0.0088, 0.0211]])
item_factors.weight tensor([[0.0038, 0.0404, 0.0192,  ..., 0.0202, 0.0041, 0.0264],
        [0.0405, 0.0069, 0.0312,  ..., 0.0078, 0.0252, 0.0100],
        [0.0201, 0.0064, 0.0136,  ..., 0.0348, 0.0444, 0.0094],
        ...,
        [0.0157, 0.0274, 0.0356,  ..., 0.0289, 0.0093, 0.0166],
        [0.0164, 0.0247, 0.0496,  ..., 0.0298, 0.0156, 0.0026],
        [0.0225, 0.0219, 0.0166,  ..., 0.0370, 0.0385, 0.0056]])


In [28]:
from tqdm.notebook import tqdm

for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
            x = x.long()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()

    # Kontrollera om losses inte är tom
    if losses:
        print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))
    else:
        print("iter #{}".format(it), "Loss: No losses calculated")       

  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: No losses calculated
iter #1 Loss: No losses calculated
iter #2 Loss: No losses calculated
iter #3 Loss: No losses calculated
iter #4 Loss: No losses calculated
iter #5 Loss: No losses calculated
iter #6 Loss: No losses calculated
iter #7 Loss: No losses calculated


KeyboardInterrupt: 

In [24]:
if losses:  # Kontrollera om losses inte är tom
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))
else:
    print("iter #{}".format(it), "Loss: No losses calculated")  # Hantera fallet när losses är tom


iter #0 Loss: No losses calculated


Jag ser att jag behöver splita cellerna så att i genders och film titel delas upp.

In [ ]:
df_movies = df_movies.assign(year=df_movies ['title'].str.extract(r'\((\d{4})\)'), title=df_movies ['title'].str.replace(r'\(\d{4}\)', '').str.strip())

df_movies['genres'] = df_movies['genres'].str.split('|')
df_movies= df_movies.explode('genres')

df_movies.head()


,movieId,title,genres,year
0,1,Toy Story (1995),Adventure,1995
0,1,Toy Story (1995),Animation,1995
0,1,Toy Story (1995),Children,1995
0,1,Toy Story (1995),Comedy,1995
0,1,Toy Story (1995),Fantasy,1995


b) Vilka är de 10 filmerna med flest ratings?

In [ ]:
# Utför sammanslagning samt kolumner samt återställt indexet till 'movieId'
#merged_df = pd.merge(df_ratings, df_movies, on='movieId')[['movieId', 'title', 'genres', 'rating', 'timestamp']]
#merged_df = merged_df.set_index('movieId').reset_index()

# Gruppera efter filmtitel och räkna antalet betyg för varje film
#top_movies = merged_df.groupby('title').size().reset_index(name='rating_count')

# Sortera resultaten i fallande ordning baserat på antalet betyg och välj de 10 första raderna
#top_10_movies = top_movies.sort_values(by='rating_count', ascending=False).head(10)

#print(top_10_movies)


c) Beräkna den genomsnittliga ratingen för dessa 10 filmerna med flest ratings.

In [ ]:
avg_rating_top_10 = top_10_movies.merge(merged_df, on='title').groupby('title')['rating'].mean()

print(avg_rating_top_10)

title
Fight Club (1999)                   4.236019
Forrest Gump (1994)                 4.068189
Inception (2010)                    4.176187
Jurassic Park (1993)                3.689013
Lion King, The (1994)               3.833477
Matrix, The (1999)                  4.160631
Pulp Fiction (1994)                 4.191778
Shrek (2001)                        3.748595
Silence of the Lambs, The (1991)    4.150287
Toy Story (1995)                    3.893508
Name: rating, dtype: float64


d) Gör en plot över årtal och antalet filmer representerade i datasetet.

Jag tar filtrerar ut så att ploten bara visar filmer från 1950 och framåt

In [ ]:
print(merged_df.head())


   movieId             title     genres  rating   timestamp
0        1  Toy Story (1995)  Adventure     4.0  1225734739
1        1  Toy Story (1995)  Animation     4.0  1225734739
2        1  Toy Story (1995)   Children     4.0  1225734739
3        1  Toy Story (1995)     Comedy     4.0  1225734739
4        1  Toy Story (1995)    Fantasy     4.0  1225734739


In [ ]:
print(merged_df['timestamp'].describe())

count    9.211335e+07
mean     1.277818e+09
std      2.529235e+08
min      7.896520e+08
25%      1.060876e+09
50%      1.284173e+09
75%      1.500351e+09
max      1.689843e+09
Name: timestamp, dtype: float64




e) Gör en plot över antalet ratings mot movieId.

f) Beräkna genomsnittliga ratings för de top 10 filmerna med flest ratings. Gör ett stapeldiagram över
dessa.

1.2 Skapa gles matris

Likt i videon i uppgift 1.0 skapade du en pivottabell av dataframet med index: "movieId", columns: "userId"
och values: "ratings". Denna pivottabell är dock "dyr" att skapa och förmodligen kommer inte din dator att
klara av skapa den om du inte filtrerar bort viss data. Fundera ut ett lämpligt sätt att filtrera ditt dataset,
pröva dig fram och motivera.#

Skapa en gles (sparse) matris av denna pivottabell mha scipy.sparse.csc_matrix(). Vill du använda
dig av all data går det också att lösa, men du behöver lösa hur du skapar den glesa matrisen utan pandas
pivot-tabell.

 1.3 Recommender system

Skapa ett recommender system med KNN och låt systemet ta input från användaren och skriva ut top 5
rekommenderade filmerna, baserat på användarens sökquery. Observera att det finns ett logiskt fel i
videon som gör att rekommendationerna inte blir så bra, försök hitta felet och åtgärda det.

a) Beskriv med ord hur ditt system fungerar.

b) Leta online och läs vidare om rekommenderarsystem och beskriv kort hur dem fungerar. Glöm inte
källhänvisa.

